# Accuinsight+ Sandbox 시작하기: 아이템 기반 협업 필터링(item based collaborative filtering)

목차  

1. [데이터 가져오기](#데이터-가져오기)
2. [모델 작성](#모델-작성)
3. [데이터 준비](#데이터-준비)
4. [모델 훈련](#모델-훈련)
5. [모델 배포 테스트](#모델-배포-테스트)

이 문서는 Accuinsight+ 3.0 Sandbox의 Scikit-learn 노트북 예제 파일입니다.  
본 예제에서는 movielens tabular 데이터를 이용하여 item based collaborative filtering 알고리즘 기반으로 아이템 간 유사도를 계산하고 Accuinsight+ 3.0에 배포 할 준비를 해보겠습니다. 

## 데이터 가져오기

https://www.kaggle.com/sengzhaotoo/movielens-small 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
rating_data = pd.read_csv('./movie_lens/ratings.csv')
movie_data = pd.read_csv('./movie_lens/movies.csv')

In [3]:
rating_data.head(2)

userId  movieId  rating   timestamp
0       1       31     2.5  1260759144
1       1     1029     3.0  1260759179

In [14]:
movie_data.head(5)

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

---
## 모델 작성
아래 세 가지의 파일을 통해 사용할 모델을 직접 작성할 수 있습니다.     
아래 `model.py`, `preprocessor.py`, `postprocessor.py` 링크를 클릭해서 해당 파일들을 편집할 수 있습니다.   
단, 작성중인 ipynb 파일의 위치가 `/home/work` 가 아닌 경우에는 이 셀을 편집하여 파일의 링크를 `/home/work` 에서 접근 가능한 상대 경로로 수정하여야 합니다.   
예를 들어, 만약 첫 화면에서 `my_notebooks` 라는 폴더를 생성하고 해당 위치에서 ipynb 파일을 작성한다면 `model.py` 의 링크를 `../` 로 시작하도록 수정합니다.   

* [`model.py`](./airuntime/sklearn/sklearnserver/trainer/model.py) 실제 사용되는 모델 파일입니다. 아래 build, save, load 세 함수를 편집하면 됩니다. 
  * `build`
  * `save`
  * `load`
* [`preprocessor.py`](./airuntime/sklearn/sklearnserver/trainer/preprocessor.py) 전처리 코드입니다. Workflow에서 전처리 노드를 작업한 경우 해당 코드가 들어있습니다. 
* [`postprocessor.py`](./airuntime/sklearn/sklearnserver/trainer/postprocessor.py) 후처리 코드입니다. 모델의 예측 결과를 후처리하는 코드를 작성합니다. 

---
## 데이터 준비 

In [5]:
rating_data.drop('timestamp', axis = 1, inplace=True)
user_movie_rating = pd.merge(rating_data, movie_data, on = 'movieId')
movie_user_rating = user_movie_rating.pivot_table('rating', index = 'title', columns='userId')
movie_user_rating.fillna(0, inplace = True)
item_based_collabor = cosine_similarity(movie_user_rating)
item_based_collabor = pd.DataFrame(data = item_based_collabor, index = movie_user_rating.index, columns = movie_user_rating.index)
item_based_collabor.head()

title                                    "Great Performances" Cats (1998)  \
title                                                                       
"Great Performances" Cats (1998)                                 1.000000   
$9.99 (2008)                                                     0.000000   
'Hellboy': The Seeds of Creation (2004)                          0.000000   
'Neath the Arizona Skies (1934)                                  0.164399   
'Round Midnight (1986)                                           0.020391   

title                                    $9.99 (2008)  \
title                                                   
"Great Performances" Cats (1998)                  0.0   
$9.99 (2008)                                      1.0   
'Hellboy': The Seeds of Creation (2004)           0.0   
'Neath the Arizona Skies (1934)                   0.0   
'Round Midnight (1986)                            0.0   

title                                    'Hellboy': The Seeds of Creation (2004)  \
title                                                                              
"Great Performances" Cats (1998)                                             0.0   
$9.99 (2008)                                                                 0.0   
'Hellboy': The Seeds of Creation (2004)                                      1.0   
'Neath the Arizona Skies (1934)                                              0.0   
'Round Midnight (1986)                                                       0.0   

title                                    'Neath the Arizona Skies (1934)  \
title                                                                      
"Great Performances" Cats (1998)                                0.164399   
$9.99 (2008)                                                    0.000000   
'Hellboy': The Seeds of Creation (2004)                         0.000000   
'Neath the Arizona Skies (1934)                                 1.000000   
'Round Midnight (1986)                                          0.124035   

title                                    'Round Midnight (1986)  \
title                                                             
"Great Performances" Cats (1998)                       0.020391   
$9.99 (2008)                                           0.000000   
'Hellboy': The Seeds of Creation (2004)                0.000000   
'Neath the Arizona Skies (1934)                        0.124035   
'Round Midnight (1986)                                 1.000000   

title                                    'Salem's Lot (2004)  \
title                                                          
"Great Performances" Cats (1998)                         0.0   
$9.99 (2008)                                             0.0   
'Hellboy': The Seeds of Creation (2004)                  1.0   
'Neath the Arizona Skies (1934)                          0.0   
'Round Midnight (1986)                                   0.0   

title                                    'Til There Was You (1997)  \
title                                                                
"Great Performances" Cats (1998)                          0.014046   
$9.99 (2008)                                              0.000000   
'Hellboy': The Seeds of Creation (2004)                   0.000000   
'Neath the Arizona Skies (1934)                           0.085436   
'Round Midnight (1986)                                    0.010597   

title                                    'burbs, The (1989)  \
title                                                         
"Great Performances" Cats (1998)                   0.000000   
$9.99 (2008)                                       0.079474   
'Hellboy': The Seeds of Creation (2004)            0.217357   
'Neath the Arizona Skies (1934)                    0.000000   
'Round Midnight (1986)                             0.143786   

title                                    'night Mother (1986)  \
title                

In [6]:
def get_item_based_collabor(title):
    return item_based_collabor[title].sort_values(ascending=False)[:6]

In [7]:
get_item_based_collabor('Godfather, The (1972)')

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.773685
Goodfellas (1990)                            0.620349
One Flew Over the Cuckoo's Nest (1975)       0.568244
American Beauty (1999)                       0.557997
Star Wars: Episode IV - A New Hope (1977)    0.546750
Name: Godfather, The (1972), dtype: float64

### 아이템 간 유사도 테이블 저장

In [8]:
import os

dirpath = '/home/work/airuntime/sklearn/models'
filename = 'item_based_collabor'
item_based_collabor.to_pickle(os.path.join(dirpath, filename))

---
## 모델 훈련

앞서 준비한 데이터로 모델을 훈련합니다. 

In [9]:
%load_ext autoreload
%autoreload 2

from airuntime.sklearn import sklearnserver as modelserver

/opt/conda/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


### 모듈을 다시 로드

In [37]:
import importlib
importlib.reload(modelserver)

<module 'airuntime.sklearn.sklearnserver' from '/home/work/airuntime/sklearn/sklearnserver/__init__.py'>

In [38]:
model = modelserver.trainer.model.SKLearnModel(dirpath=dirpath)

### 모델 예측 테스트

모델 훈련 후 예측이 가능한지 테스트합니다.

In [35]:
dataset = pd.DataFrame({
    "movie_name": movie_data.title[:5].values.tolist(),
    "top_n": [10 for i in range(5)]
})
dataset

movie_name  top_n
0                    Toy Story (1995)     10
1                      Jumanji (1995)     10
2             Grumpier Old Men (1995)     10
3            Waiting to Exhale (1995)     10
4  Father of the Bride Part II (1995)     10

In [39]:
model.predict(dataset)

[[('Toy Story 2 (1999)', 0.59),
  ('Star Wars: Episode IV - A New Hope (1977)', 0.58),
  ('Forrest Gump (1994)', 0.56),
  ('Independence Day (a.k.a. ID4) (1996)', 0.56),
  ('Groundhog Day (1993)', 0.55),
  ('Back to the Future (1985)', 0.54),
  ('Jurassic Park (1993)', 0.54),
  ('Shrek (2001)', 0.53),
  ('Star Wars: Episode VI - Return of the Jedi (1983)', 0.53),
  ('Pulp Fiction (1994)', 0.53)],
 [('Mrs. Doubtfire (1993)', 0.55),
  ('Lion King, The (1994)', 0.53),
  ('Santa Clause, The (1994)', 0.51),
  ('Mask, The (1994)', 0.49),
  ('Beauty and the Beast (1991)', 0.49),
  ('Aladdin (1992)', 0.49),
  ('Ace Ventura: Pet Detective (1994)', 0.48),
  ('Ghost (1990)', 0.47),
  ('Jurassic Park (1993)', 0.47),
  ('Speed (1994)', 0.47)],
 [('Phenomenon (1996)', 0.51),
  ('Multiplicity (1996)', 0.47),
  ('Executive Decision (1996)', 0.45),
  ('Nutty Professor, The (1996)', 0.44),
  ('Sabrina (1995)', 0.44),
  ('Independence Day (a.k.a. ID4) (1996)', 0.43),
  ('Rock, The (1996)', 0.41),
  ('Twi

### 모델 불러오기 테스트

배포 전에 모델 저장이 잘 수행되었는 지 확인합니다

In [40]:
loaded_model = modelserver.trainer.model.SKLearnModel(dirpath=dirpath)

### 저장한 모델 예측 테스트

예측결과가 저장한 후 모델의 예측 결과와 동일한 지 확인합니다

In [43]:
np.array_equal(model.predict(dataset), loaded_model.predict(dataset))

True

---
## 모델 배포 테스트

모델을 정상적으로 배포가능한 지 테스트용 서버를 실행하여 API 테스트를 수행하겠습니다.

### 테스트용 서버 실행

테스트용 서버 ui를 실행합니다.  
아래 코드를 실행한 뒤, "SELECT" 버튼을 눌러서 모델 생성에 사용된 데이터 파일을 지정합니다.  
그리고 Target Col 에 Target이 될 컬럼명   
variety  
를 입력한 뒤 Test Server: Start를 클릭합니다

In [45]:
%run -i /home/work/airuntime/_widgets/run_server_ui.py -k sklearn

[I 230220 10:48:13 services:1727] object_store_memory is not verified when plasma_directory is set.


FileChooser(path='/home/work/dataset', filename='dataset', title='Select Dataset from <b>`dataset`</b> directo…

### 모델 예측 요청 전달

실행한 테스트용 서버에 예측 요청이 정상적으로 전달되고 결과가 나오는지 확인합니다.

In [46]:
%run -i /home/work/airuntime/_widgets/request_ui.py